## Section 5: We use the two new profiles, the concept maps, and the two outlines to generate two interviews/lessons that explore and delve into the initial question from two different perspectives — Max's and Samanta's — with an educational and informative approach

In [2]:
#versione con max che cambia in base al prompt 
import gradio as gr
import ollama
import os
import json
from pathlib import Path
import subprocess
from time import sleep
# Variabili globali

sintesi_raccolte = []
progetto=[]
messages = []  # Lista per memorizzare i messaggi scambiati
turno = 0      # Contatore per i turni
domanda_osservatore = None  # Domanda dell'osservatore


modello_max='gemma3:12b'
modello_sam='gemma3:12b'
modello_pol='gemma3:12b'


def clean_ollama_sessions():
    """Versione per Jupyter"""
    try:
        output = !ollama kill 2>&1
        if "no running sessions" in str(output).lower():
            print("✅ Nessuna sessione attiva da terminare")
        else:
            print("🛑 Sessioni terminate con successo")
    except:
        print("❌ Comando ollama non trovato")

# Esegui all'inizio del notebook
clean_ollama_sessions()


css = """
button:active {
    background-color: #ff6600 !important;  /* colore di sfondo quando premuto */
    color: white !important;               /* colore del testo quando premuto */
    transform: scale(0.97);                /* leggero effetto di "pressione" */
}
"""

# Carica il percorso salvato
CONFIG_PATH = Path(os.getenv("APPDATA")) / "MaGo" / "config.json"

def load_project_path():
    """Carica il percorso dal file di configurazione"""
    if not CONFIG_PATH.exists():
        return None
    
    with open(CONFIG_PATH, "r") as f:
        config = json.load(f)
    return config.get("project_path")

# Verifica all'avvio
#project_path = load_project_path()
# 🔥 QUI: cast a Path
project_path = Path(load_project_path())  # <-- Fondamentale!

if not project_path.exists():
    raise RuntimeError("Percorso non valido!")

os.chdir(project_path)

if not project_path or not Path(project_path).exists():
    raise RuntimeError(
        "🛑 Percorso non configurato o non valido!\n"
        "Esegui prima il notebook 'installer.ipynb' e imposta la cartella corretta."
    )

# Imposta la directory di lavoro
os.chdir(project_path)
print(f"✔️ Directory di lavoro impostata a: {project_path}")


global project_path


# Variabile per salvare lo scenario selezionato
scena_selezionata = gr.State(value=None)

def aggiorna_scena(scelta):
    return scelta  # Restituisce il valore selezionato

messages = []  # Lista dei messaggi globali

def interazione_osservatore(domanda, voto):
    global messages
    messages.append({'role': 'user', 'content': f"Domanda dall'osservatore: {domanda}"})

    if voto < 3:
        azione = "approfondire"
    else:
        azione = "concludere"

    messages.append({'role': 'system', 'content': f"Decisione: {azione}"})

    return azione

def inizializza_lezione_max(prof1, domanda, scaletta,  map1,lingua):
   

    messages = [
    {'role': 'system','content': f"Tu sei Max, un docente  esperto in '{prof1}'. Tu sei Max  e hai solide competenze e sai spiegare concetti complessi in modo semplice, ordinato e coinvolgente. "},
     {'role': 'system','content':f"Tu Max sei un bravo divulgatore scientifico: chiaro, diretto, logico.Hai 20 anni di esperienza di insegnamento e parli davanti a studenti motivati che ti seguono con grande attenzione e interesse" },
     {'role': 'system','content':f" Il tuo compito Max è di approfondire questo tema «{domanda}» rispondendo alle domande che ti vengono poste in modo esteso, divulgativo e brillante, ma soprattutto appassionato  " },
     {'role': 'system','content':f"Tu Pol  sei un moderatore e devi intervistare Max. Tu Pol per fare le domande a Max hai preparato una traccia '{scaletta}' " }, 
     {'role': 'system','content':f" Il tuo compito Pol è quello di  prendere ogni punto della traccia e formulare una domanda per Max ed invitarlo a rispondere in modo chiaro, esauriente ed esteso. Pol se vuoi sottolineare qualche aspetto della risposta di Max lo puoi fare" },
     {'role': 'system','content':f"Tu Pol Alla fine della lezione dopo avere posto a Max tutte le domande contenute nella scaletta fai un breve riassunto e termina la lezione " },
     {'role': 'system','content':f"Max, inizia la lezione  dopo avere ringraziato il gruppo di studio Ma&Go che ti ha invitato a tenere questa lezione"},
    {'role': 'system','content':f" Attenzione la tua risposta deve essere data nella lingua '{lingua}'"}
]

    return messages

def inizializza_lezione_sam(prof2, domanda,scal2,map2,lingua):
    
 
    messages = [
    {'role': 'system','content': f"Tu sei Samanta, una docente  esperta in '{prof2}'. Tu Samanta hai solide competenze e sai spiegare concetti complessi in modo semplice, ordinato e coinvolgente. "},
     {'role': 'system','content':f"Tu Samanta sei un bravo divulgatore scientifico: chiaro, diretto, logico. Hai 20 anni di esperienza di insegnamento e parli davanti a studenti motivati che ti seguono con grande attenzione e interesse" },
     {'role': 'system','content':f" Il tuo compito Samanta è di approfondire questo tema «{domanda}» rispondendo alle domande che ti vengono poste in modo esteso, divulgativo e brillante " },
     {'role': 'system','content':f"Tu Pol  sei un moderatore del gruppo di studio Ma&Go e sei un grande esperto dei temi contenuti nella  «{domanda}» e devi intervistare Samanta. Tu Pol per fare le domande a Samanta hai preparato una traccia '{scal2}' " }, 
     {'role': 'system','content':f"Il tuo compito Pol è quello di  prendere ogni punto della traccia. Tu Pol devi rispettare l'ordine dei punti contenuti nella scaletta  e formulare una domanda per Samanta ed invitarla a rispondere in modo chiaro, esauriente ed esteso. " },
     {'role': 'system','content':f"Tu Pol Alla fine della lezione dopo avere posto a Samanta tutte le domande contenute nella scaletta fai un breve riassunto e termina la lezione." },
       {'role': 'system','content':f" Attenzione tu Pol e tu Samanta dovete esprimervi nella lingua '{lingua}'"}
]

   
    # Qui puoi restituire o processare il messaggio
    return messages



# Funzione base migliorata
def chat_lezione_max(prof1, domanda, scal1, map1, lingua, progetto):
    global project_path
    
    project_dir = project_path / "progetti" / progetto
    chat_stato = []
    messages=[]
    
    messages = inizializza_lezione_max(prof1, domanda, scal1,map1,lingua)
    punti_scaletta = [riga.strip() for riga in scal1.strip().split("\n") if riga.strip()]

    print(f"\n🎤 Pol: Iniziamo la lezione su: {domanda}\n")

    for punto in punti_scaletta:
        numero, testo = punto.split(".", 1)
        numero = numero.strip()
        titolo = testo.strip()
        print(f"\n📍 Punto {numero}: {titolo}")
        
        # 1. Pol genera la domanda
        prompt_pol = f"""Sei Pol, il moderatore. Formula una domanda in {lingua} chiara e stimolante su:
        {titolo}
        Rispondi solo con la domanda."""
        
        response_pol = ollama.chat(
            model=modello_pol,
            messages=[{'role': 'user', 'content': prompt_pol}]
        )
        domanda_pol = response_pol['message']['content']
        chat_stato.append((f"🤖 Pol", domanda_pol))
        print(f"🤖 Pol: {domanda_pol}")

        # 2. Samanta risponde
        prompt_max = f"""Sei Max, docente esperto. Rispondi in {lingua} in modo chiaro e autorevole a:
        {domanda_pol}
        - Sii conciso
        - Fornisci esempi concreti
        - Non ripetere la domanda"""
        
        response_max = ollama.chat(
            model=modello_max,
            messages=[{'role': 'user', 'content': prompt_max}],
            options={"temperature": 0.7}
        )
        risposta_max = response_max['message']['content']
        chat_stato.append((f"🤖 Max", risposta_max))
        print(f"🤖 Max: {risposta_max}\n")

    # Salvataggio
    with open(project_dir / 'intervista_max.txt', 'w', encoding="utf-8") as file:
        for ruolo, messaggio in chat_stato:
            file.write(f"{ruolo}: {messaggio}\n")
    
    return chat_stato

    
def chat_lezione_sam(prof2, domanda, scal2, map2, lingua, progetto):
    global project_path
    
    project_dir = project_path / "progetti" / progetto
    chat_stato = []
    messages=[]
    
    messages = inizializza_lezione_sam(prof2, domanda, scal2,map2,lingua)
    punti_scaletta = [riga.strip() for riga in scal2.strip().split("\n") if riga.strip()]

    print(f"\n🎤 Pol: Iniziamo la lezione su: {domanda}\n")

    for punto in punti_scaletta:
        numero, testo = punto.split(".", 1)
        numero = numero.strip()
        titolo = testo.strip()
        print(f"\n📍 Punto {numero}: {titolo}")
        
        # 1. Pol genera la domanda
        prompt_pol = f"""Sei Pol, il moderatore. Formula una domanda in {lingua} chiara e stimolante su:
        {titolo}
        Rispondi solo con la domanda."""
        
        response_pol = ollama.chat(
            model=modello_pol,
            messages=[{'role': 'user', 'content': prompt_pol}]
        )
        domanda_pol = response_pol['message']['content']
        chat_stato.append((f"🤖 Pol", domanda_pol))
        print(f"🤖 Pol: {domanda_pol}")

        # 2. Samanta risponde
        prompt_sam = f"""Sei Samanta, docente esperto. Rispondi in {lingua} in modo chiaro e autorevole a:
        {domanda_pol}
        - sei didattica nelle tue spiegazioni, tecnica nel linguaggio, ma  appassionata e coinvolgente 
        - Fornisci esempi concreti
        - Non ripetere la domanda"""
        
        response_sam = ollama.chat(
            model=modello_sam,
            messages=[{'role': 'user', 'content': prompt_sam}],
            options={"temperature": 0.7}
        )
        risposta_sam = response_sam['message']['content']
        chat_stato.append((f"🤖 Samanta", risposta_sam))
        print(f"🤖 Samanta: {risposta_sam}\n")

    # Salvataggio
    with open(project_dir / 'intervista_sam.txt', 'w', encoding="utf-8") as file:
        for ruolo, messaggio in chat_stato:
            file.write(f"{ruolo}: {messaggio}\n")
    
    return chat_stato
    
  #pol formula la domanda 
def interroga_moderatore_max(nome, titolo, messages, lingua):
    prompt_pol = f"""
    Sei Pol, il moderatore. Devi fare UNA domanda alla volta sul tema: «{titolo}» in {lingua}.
    - **Non concludere** l'intervista.  
    - **Non anticipare** riassunti o finali.  
    - Rispondi SOLO con la domanda, senza commenti. 
    - Non tornare mai sui punti precedenti ad esempio NON tornare sul punto 1.
    - Tu Pol puoi chiedere a Max di approfondire un argomento della sua risposta
    """
    
    messages.append({'role': 'user', 'content': prompt_pol})
    response = ollama.chat(
        model=modello_max,
        messages=messages,
        options={"temperature": 0.3}  # Imposta temperatura bassa (0.3-0.5)
    )
    domanda = response['message']['content']
    
    # Controllo anti-chiusura anticipata (semplice ma efficace)
    if "FINE" in domanda.upper() or "GRAZIE" in domanda.upper():
        domanda = "Continua con la prossima domanda sul tema."
    
    #print(f"🤖 {nome}: {domanda}\n")
    messages.append({'role': 'assistant', 'content': domanda})
    return domanda, messages



def interroga_attore_max(nome, messages,lingua):
    prompt_max = f"""
    Sei Max, rispondi alla domanda di Pol in {lingua}.
    - **Non concludere** l'intervista.  
    - Sei un docente molto apprezzato rispondi alla la domanda di Pol in modo appassionato e autorevole.  
    """
    messages.append({'role': 'user', 'content': prompt_max})
    response = ollama.chat(model=modello_max, messages=messages)
    risposta = response['message']['content']
    print(f"🤖 {nome}: {risposta}\n")

    messages.append({'role': 'assistant', 'content': risposta})
    return risposta, messages

    
def aggiorna_progetti():
    """Restituisce la lista delle cartelle dei progetti."""
    global project_path
    project_dir = Path(project_path) / "progetti" 
    if not os.path.exists(project_dir):
        os.makedirs(project_dir)  # Crea la directory se non esiste
    return [d for d in os.listdir(project_dir) if os.path.isdir(os.path.join(project_dir, d))]

def carica_progetto(progetto):
    
    global project_path

    """Carica i file prof1.txt, prof2.txt, scal1.txt, scal2.txt e domanda.txt dalla cartella selezionata."""
 
    project_dir = project_path / "progetti" /progetto  #
    
    if not os.path.exists(project_dir):
        return "Errore: Progetto non trovato", "", "", "", ""
    if not os.path.exists(project_dir):
        return "Errore: Progetto non trovato", "", "", "", ""

    def leggi_file(nome_file):
        path = os.path.join(project_dir, nome_file)
        try:
            with open(path, 'r', encoding='utf-8') as f:
                contenuto = f.read().strip()
                return contenuto if contenuto else ""  # Restituisce stringa vuota se il file è vuoto
        except FileNotFoundError:
            print(f"Attenzione: il file '{nome_file}' non è stato trovato.")
            return ""
        except Exception as e:
            print(f"Errore nella lettura del file '{nome_file}': {e}")
            return ""

    domanda = leggi_file("domanda.txt")
    prof1 = leggi_file("prof1.txt")
    prof2 = leggi_file("prof2.txt")
    map1 = leggi_file("map1.txt")
    map2 = leggi_file("map2.txt")
    scal1 = leggi_file("scal1.txt")
    scal2 = leggi_file("scal2.txt")
    lingua= leggi_file("lingua.txt")
   
    return prof1, prof2, domanda, scal1, scal2,map1,map2,lingua
    
def invia_domanda(domanda, messages):
    global domanda_osservatore
    domanda_osservatore=domand
    if domanda.strip():
        messages.append(("Osservatore", domanda))  # Aggiunge la domanda alla chat
    return messages  # Ritorna l'output aggiornato per essere mostrato
    
def avvia_interfaccia():
    with gr.Blocks(css=css) as demo:
        stile_scelto = gr.State([]) 
        gr.Markdown("# Generate Max and Samanta's Interviews/Lessons")
        def carica_scenario(scelta):
         
            # Qui puoi fare qualcosa con il valore numerico scelto, come caricare un prompt o avviare una discussione
            return f" {scelta}"
        # Variabile per salvare lo scenario selezionato
        scena_selezionata = gr.State(value=None)   
        map1_output= gr.State(value=None) 
        map2_output= gr.State(value=None) 
        
        with gr.Row():
            Progetto = gr.Dropdown(label="Select a project", choices=aggiorna_progetti())
            btn_carica = gr.Button("Load project")

        
        domanda_output = gr.Textbox(label="QUESTION", interactive=False)
        with gr.Row():
             prof1_output = gr.Textbox(label="MAX's Profile", interactive=False)
             prof2_output = gr.Textbox(label="Samanta's Profile", interactive=False)
           # with gr.Column():
        with gr.Row():
             scal1_output = gr.Textbox(label="MAX's Outline", interactive=False)
             scal2_output = gr.Textbox(label="Samanta's Outline", interactive=False)


        lingua = gr.State()
     
        btn_carica.click(fn=carica_progetto, inputs=[Progetto], 
        outputs=[prof1_output, prof2_output, domanda_output,scal1_output,scal2_output,map1_output,map2_output,lingua])

                
        with gr.Row():
            with gr.Column(scale=1):
                btn_lezione_max = gr.Button("Max's Lesson")
            with gr.Column(scale=1):  
                btn_lezione_sam = gr.Button("Samanta's Lesson")
        #gr.Textbox(label="Output della lezione", lines=10, interactive=False)

                 
       
        chat_stato = gr.State([])  # Stato interno della chat
        lezione_output_textbox = gr.Textbox(label="Lesson Output", lines=10, interactive=False)
     
        btn_lezione_max.click(
        fn=chat_lezione_max,
        inputs=[prof1_output, domanda_output, scal1_output, map1_output,lingua,Progetto],
        outputs=lezione_output_textbox
        )
     
        btn_lezione_sam.click(
        fn=chat_lezione_sam,
        inputs=[prof2_output, domanda_output, scal2_output, map2_output,lingua,Progetto],
        outputs=lezione_output_textbox
        )

       
    demo.launch()

if __name__ == "__main__":
    avvia_interfaccia()

🛑 Sessioni terminate con successo
✔️ Directory di lavoro impostata a: C:\Users\maian\notebook\py10\MaGo
* Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.



🎤 Pol: Iniziamo la lezione su: Il tempo è una realtà oggettiva o una costruzione della mente?


📍 Punto 1: Tempo: Realtà Oggettiva o Costruzione Mentale?
🤖 Pol: Il tempo, come lo percepiamo e misuriamo, è una realtà fisica indipendente dalla nostra mente, o una costruzione puramente concettuale, un modo per dare senso all'esperienza?

🤖 Max: Come docente esperto, posso affermare che il tempo è un fenomeno complesso che si colloca a cavallo tra realtà fisica e costruzione concettuale.

**In termini fisici, il tempo esiste indipendentemente da noi.** La fisica moderna, con la relatività di Einstein, ci dice che il tempo è intrecciato con lo spazio, formando lo spaziotempo, e che la sua velocità può variare a seconda della gravità e della velocità relativa.  Ad esempio, un orologio in cima a una montagna ticchetta leggermente più velocemente di uno a livello del mare, a causa della diversa forza gravitazionale. Questo è un fatto misurabile e verificabile.

**Tuttavia, la *percezione* del